# Forest Cover Type Classification

## Task 3: Multi-class Classification using UCI Covertype Dataset

**Objective:** Predict forest cover types based on cartographic and environmental features

**Dataset:** UCI Covertype Dataset (581,012 samples, 54 features, 7 classes)

**Tools & Libraries:** Python, Pandas, Scikit-learn, XGBoost, Matplotlib, Seaborn

**Covered Topics:** Multi-class classification, Tree-based modeling, Feature importance analysis

---

### Table of Contents
1. [Data Loading and Exploration](#1-data-loading-and-exploration)
2. [Data Preprocessing](#2-data-preprocessing)
3. [Model Training and Evaluation](#3-model-training-and-evaluation)
4. [Confusion Matrix Visualization](#4-confusion-matrix-visualization)
5. [Feature Importance Analysis](#5-feature-importance-analysis)
6. [Hyperparameter Tuning](#6-hyperparameter-tuning)
7. [Model Comparison](#7-model-comparison)
8. [Results and Conclusions](#8-results-and-conclusions)


## 1. Data Loading and Exploration

Let's start by importing the necessary libraries and loading the dataset.


In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import warnings
warnings.filterwarnings('ignore')

# Set style for better plots
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Libraries imported successfully!")
print("Ready to start the forest cover type classification project!")


In [ ]:
# Load the dataset
print("Loading Forest Cover Type dataset...")
data = pd.read_csv('covtype.data.gz', header=None, compression='gzip')

# Define feature names based on the dataset documentation
feature_names = [
    'Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology',
    'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
    'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
    'Horizontal_Distance_To_Fire_Points'
]

# Add wilderness area and soil type columns
wilderness_areas = [f'Wilderness_Area_{i}' for i in range(1, 5)]
soil_types = [f'Soil_Type_{i}' for i in range(1, 41)]
feature_names.extend(wilderness_areas)
feature_names.extend(soil_types)

# Set column names
columns = feature_names + ['Cover_Type']
data.columns = columns

print(f"Dataset loaded successfully!")
print(f"Shape: {data.shape}")
print(f"Features: {len(feature_names)}")
print(f"Target classes: 7")

# Display first few rows
data.head()


In [ ]:
# Forest cover type names
cover_types = {
    1: 'Spruce/Fir',
    2: 'Lodgepole Pine', 
    3: 'Ponderosa Pine',
    4: 'Cottonwood/Willow',
    5: 'Aspen',
    6: 'Douglas-fir',
    7: 'Krummholz'
}

# Basic dataset information
print("="*50)
print("DATASET INFORMATION")
print("="*50)
print(f"Total samples: {len(data):,}")
print(f"Total features: {len(feature_names)}")
print(f"Missing values: {data.isnull().sum().sum()}")

# Class distribution
print(f"\nClass Distribution:")
class_counts = data['Cover_Type'].value_counts().sort_index()
for cover_type, count in class_counts.items():
    percentage = (count / len(data)) * 100
    print(f"{cover_type} ({cover_types[cover_type]}): {count:,} ({percentage:.2f}%)")

# Statistical summary of quantitative features
print(f"\nStatistical Summary of Quantitative Features:")
quantitative_features = feature_names[:10]  # First 10 are quantitative
data[quantitative_features].describe()


In [ ]:
# Visualize class distribution and key features
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Class distribution bar plot
axes[0, 0].bar(class_counts.index, class_counts.values, color='skyblue', edgecolor='black')
axes[0, 0].set_title('Forest Cover Type Distribution', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Cover Type')
axes[0, 0].set_ylabel('Count')
axes[0, 0].set_xticks(class_counts.index)
axes[0, 0].set_xticklabels([f'{i}\n({cover_types[i]})' for i in class_counts.index], rotation=45)

# Class distribution pie chart
axes[0, 1].pie(class_counts.values, labels=[cover_types[i] for i in class_counts.index], 
               autopct='%1.1f%%', startangle=90)
axes[0, 1].set_title('Cover Type Distribution (Percentage)', fontsize=14, fontweight='bold')

# Elevation distribution by cover type
for cover_type in sorted(data['Cover_Type'].unique()):
    subset = data[data['Cover_Type'] == cover_type]['Elevation']
    axes[1, 0].hist(subset, alpha=0.6, label=f'{cover_type} ({cover_types[cover_type]})', bins=30)
axes[1, 0].set_xlabel('Elevation (meters)')
axes[1, 0].set_ylabel('Frequency')
axes[1, 0].set_title('Elevation Distribution by Cover Type', fontsize=14, fontweight='bold')
axes[1, 0].legend(bbox_to_anchor=(1.05, 1), loc='upper left')

# Aspect distribution
axes[1, 1].hist(data['Aspect'], bins=50, color='lightgreen', edgecolor='black', alpha=0.7)
axes[1, 1].set_xlabel('Aspect (degrees)')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].set_title('Aspect Distribution', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()


I

In [ ]:
# Separate features and target
X = data.drop('Cover_Type', axis=1)
y = data['Cover_Type']

print("="*50)
print("DATA PREPROCESSING")
print("="*50)
print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")

# Check for missing values
missing_values = X.isnull().sum()
if missing_values.sum() > 0:
    print(f"Missing values found: {missing_values.sum()}")
    X = X.fillna(X.median())
else:
    print("No missing values found.")

# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training set: {X_train.shape[0]:,} samples")
print(f"Test set: {X_test.shape[0]:,} samples")

# Scale the features (only quantitative features)
scaler = StandardScaler()
quantitative_features = feature_names[:10]

X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()

X_train_scaled[quantitative_features] = scaler.fit_transform(X_train[quantitative_features])
X_test_scaled[quantitative_features] = scaler.transform(X_test[quantitative_features])

print("Data preprocessing completed!")


## 3. Model Training and Evaluation

Let's train multiple classification models and evaluate their performance.


In [ ]:
# Define models
models = {
    'Random Forest': RandomForestClassifier(
        n_estimators=100, 
        random_state=42, 
        n_jobs=-1,
        class_weight='balanced'
    ),
    'XGBoost': xgb.XGBClassifier(
        n_estimators=100,
        random_state=42,
        eval_metric='mlogloss',
        use_label_encoder=False
    ),
    'Logistic Regression': LogisticRegression(
        random_state=42,
        max_iter=1000,
        class_weight='balanced'
    ),
    'SVM': SVC(
        random_state=42,
        class_weight='balanced',
        probability=True
    )
}

# Train models and store results
results = {}

print("="*50)
print("MODEL TRAINING")
print("="*50)

for name, model in models.items():
    print(f"\nTraining {name}...")
    
    # Use scaled data for models that benefit from scaling
    if name in ['Logistic Regression', 'SVM']:
        X_train_use = X_train_scaled
        X_test_use = X_test_scaled
    else:
        X_train_use = X_train
        X_test_use = X_test
    
    # Train the model
    model.fit(X_train_use, y_train)
    
    # Make predictions
    y_pred = model.predict(X_test_use)
    
    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    
    # Store results
    results[name] = {
        'model': model,
        'predictions': y_pred,
        'accuracy': accuracy
    }
    
    print(f"{name} - Accuracy: {accuracy:.4f}")

print("\nModel training completed!")


In [ ]:
# Detailed evaluation of all models
print("="*50)
print("MODEL EVALUATION")
print("="*50)

evaluation_results = {}

for name, result in results.items():
    print(f"\n{name} - Detailed Evaluation:")
    print("-" * 40)
    
    y_pred = result['predictions']
    
    # Classification report
    report = classification_report(
        y_test, y_pred, 
        target_names=[cover_types[i] for i in sorted(cover_types.keys())],
        output_dict=True
    )
    
    # Store results
    evaluation_results[name] = {
        'accuracy': result['accuracy'],
        'classification_report': report
    }
    
    # Print detailed report
    print(classification_report(
        y_test, y_pred,
        target_names=[cover_types[i] for i in sorted(cover_types.keys())]
    ))

# Create comparison table
print(f"\n{'Model':<20} {'Accuracy':<10} {'Macro Avg F1':<15} {'Weighted Avg F1':<15}")
print("-" * 60)

for name, result in evaluation_results.items():
    accuracy = result['accuracy']
    macro_f1 = result['classification_report']['macro avg']['f1-score']
    weighted_f1 = result['classification_report']['weighted avg']['f1-score']
    print(f"{name:<20} {accuracy:<10.4f} {macro_f1:<15.4f} {weighted_f1:<15.4f}")


## 4. Confusion Matrix Visualization

Let's visualize the confusion matrices for all models to understand their performance better.


In [ ]:
# Plot confusion matrices for all models
print("="*50)
print("CONFUSION MATRICES")
print("="*50)

n_models = len(results)
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.ravel()

for idx, (name, result) in enumerate(results.items()):
    y_pred = result['predictions']
    
    # Create confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    
    # Plot confusion matrix
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
               xticklabels=[cover_types[i] for i in sorted(cover_types.keys())],
               yticklabels=[cover_types[i] for i in sorted(cover_types.keys())],
               ax=axes[idx])
    
    axes[idx].set_title(f'{name}\nAccuracy: {result["accuracy"]:.4f}', 
                       fontsize=12, fontweight='bold')
    axes[idx].set_xlabel('Predicted')
    axes[idx].set_ylabel('Actual')
    axes[idx].tick_params(axis='x', rotation=45)
    axes[idx].tick_params(axis='y', rotation=0)

plt.tight_layout()
plt.show()


## 5. Feature Importance Analysis

Let's analyze which features are most important for predicting forest cover types.


In [ ]:
# Feature importance analysis for tree-based models
print("="*50)
print("FEATURE IMPORTANCE ANALYSIS")
print("="*50)

# Get tree-based models
tree_models = {name: result['model'] for name, result in results.items() 
              if hasattr(result['model'], 'feature_importances_')}

if tree_models:
    n_models = len(tree_models)
    fig, axes = plt.subplots(1, n_models, figsize=(6*n_models, 8))
    
    if n_models == 1:
        axes = [axes]
    
    for idx, (name, model) in enumerate(tree_models.items()):
        # Get feature importance
        importance = model.feature_importances_
        
        # Create feature importance dataframe
        feature_importance_df = pd.DataFrame({
            'feature': feature_names,
            'importance': importance
        }).sort_values('importance', ascending=False).head(15)
        
        # Plot feature importance
        sns.barplot(data=feature_importance_df, x='importance', y='feature', 
                   ax=axes[idx], palette='viridis')
        axes[idx].set_title(f'{name} - Top 15 Features', 
                           fontsize=12, fontweight='bold')
        axes[idx].set_xlabel('Feature Importance')
        axes[idx].set_ylabel('Features')
    
    plt.tight_layout()
    plt.show()
    
    # Print top features for each model
    for name, model in tree_models.items():
        importance = model.feature_importances_
        feature_importance_df = pd.DataFrame({
            'feature': feature_names,
            'importance': importance
        }).sort_values('importance', ascending=False)
        
        print(f"\n{name} - Top 10 Most Important Features:")
        print("-" * 50)
        for i, (_, row) in enumerate(feature_importance_df.head(10).iterrows()):
            print(f"{i+1:2d}. {row['feature']:<35} {row['importance']:.4f}")
else:
    print("No tree-based models found for feature importance analysis.")


## 6. Hyperparameter Tuning

Let's perform hyperparameter tuning for the best performing models to improve their accuracy.


In [ ]:
# Hyperparameter tuning for Random Forest
print("="*50)
print("HYPERPARAMETER TUNING - RANDOM FOREST")
print("="*50)

# Define parameter grid for Random Forest
rf_param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Create Random Forest model for tuning
rf_model = RandomForestClassifier(random_state=42, n_jobs=-1, class_weight='balanced')

# Perform grid search
print("Performing grid search for Random Forest...")
rf_grid_search = GridSearchCV(
    rf_model, rf_param_grid, 
    cv=3, scoring='accuracy', n_jobs=-1, verbose=1
)

rf_grid_search.fit(X_train, y_train)

# Get best parameters and score
rf_best_params = rf_grid_search.best_params_
rf_best_score = rf_grid_search.best_score_

print(f"\nBest parameters: {rf_best_params}")
print(f"Best cross-validation score: {rf_best_score:.4f}")

# Train model with best parameters and evaluate on test set
rf_best_model = rf_grid_search.best_estimator_
rf_y_pred_tuned = rf_best_model.predict(X_test)
rf_tuned_accuracy = accuracy_score(y_test, rf_y_pred_tuned)

print(f"Test accuracy with tuned parameters: {rf_tuned_accuracy:.4f}")

# Compare with original model
rf_original_accuracy = results['Random Forest']['accuracy']
rf_improvement = rf_tuned_accuracy - rf_original_accuracy

print(f"\nComparison:")
print(f"Original accuracy: {rf_original_accuracy:.4f}")
print(f"Tuned accuracy: {rf_tuned_accuracy:.4f}")
print(f"Improvement: {rf_improvement:+.4f}")


In [ ]:
# Hyperparameter tuning for XGBoost
print("="*50)
print("HYPERPARAMETER TUNING - XGBOOST")
print("="*50)

# Define parameter grid for XGBoost
xgb_param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 6, 10],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 0.9, 1.0]
}

# Create XGBoost model for tuning
xgb_model = xgb.XGBClassifier(random_state=42, eval_metric='mlogloss', use_label_encoder=False)

# Perform grid search
print("Performing grid search for XGBoost...")
xgb_grid_search = GridSearchCV(
    xgb_model, xgb_param_grid, 
    cv=3, scoring='accuracy', n_jobs=-1, verbose=1
)

xgb_grid_search.fit(X_train, y_train)

# Get best parameters and score
xgb_best_params = xgb_grid_search.best_params_
xgb_best_score = xgb_grid_search.best_score_

print(f"\nBest parameters: {xgb_best_params}")
print(f"Best cross-validation score: {xgb_best_score:.4f}")

# Train model with best parameters and evaluate on test set
xgb_best_model = xgb_grid_search.best_estimator_
xgb_y_pred_tuned = xgb_best_model.predict(X_test)
xgb_tuned_accuracy = accuracy_score(y_test, xgb_y_pred_tuned)

print(f"Test accuracy with tuned parameters: {xgb_tuned_accuracy:.4f}")

# Compare with original model
xgb_original_accuracy = results['XGBoost']['accuracy']
xgb_improvement = xgb_tuned_accuracy - xgb_original_accuracy

print(f"\nComparison:")
print(f"Original accuracy: {xgb_original_accuracy:.4f}")
print(f"Tuned accuracy: {xgb_tuned_accuracy:.4f}")
print(f"Improvement: {xgb_improvement:+.4f}")


## 7. Model Comparison

Let's compare all models including the tuned versions.


In [ ]:
# Cross-validation analysis
print("="*50)
print("CROSS-VALIDATION ANALYSIS")
print("="*50)

cv_results = {}

for name, result in results.items():
    model = result['model']
    
    # Use appropriate data
    if name in ['Logistic Regression', 'SVM']:
        X_use = X_train_scaled
    else:
        X_use = X_train
    
    # Perform 5-fold cross-validation
    cv_scores = cross_val_score(model, X_use, y_train, cv=5, scoring='accuracy')
    
    cv_results[name] = {
        'mean_cv_score': cv_scores.mean(),
        'std_cv_score': cv_scores.std(),
        'cv_scores': cv_scores
    }
    
    print(f"{name}:")
    print(f"  CV Scores: {cv_scores}")
    print(f"  Mean CV Score: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")
    print()

# Plot cross-validation results
plt.figure(figsize=(12, 6))

model_names = list(cv_results.keys())
mean_scores = [cv_results[name]['mean_cv_score'] for name in model_names]
std_scores = [cv_results[name]['std_cv_score'] for name in model_names]

plt.bar(model_names, mean_scores, yerr=std_scores, capsize=5, 
        color='lightblue', edgecolor='black', alpha=0.7)
plt.title('Cross-Validation Scores Comparison', fontsize=14, fontweight='bold')
plt.ylabel('CV Accuracy Score')
plt.xlabel('Models')
plt.xticks(rotation=45)
plt.grid(axis='y', alpha=0.3)

# Add value labels on bars
for i, (mean, std) in enumerate(zip(mean_scores, std_scores)):
    plt.text(i, mean + std + 0.001, f'{mean:.3f}', 
            ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()


## 8. Results and Conclusions

Let's summarize our findings and provide final recommendations.


In [ ]:
# Final results summary
print("="*60)
print("COMPREHENSIVE ANALYSIS REPORT")
print("="*60)

# Dataset summary
print(f"\nDATASET SUMMARY:")
print(f"Total samples: {len(data):,}")
print(f"Features: {len(feature_names)}")
print(f"Classes: {len(cover_types)}")

# Class distribution
print(f"\nCLASS DISTRIBUTION:")
class_counts = data['Cover_Type'].value_counts().sort_index()
for cover_type, count in class_counts.items():
    percentage = (count / len(data)) * 100
    print(f"  {cover_type} ({cover_types[cover_type]}): {count:,} ({percentage:.2f}%)")

# Model performance summary
print(f"\nMODEL PERFORMANCE SUMMARY:")
print(f"{'Model':<25} {'Accuracy':<10} {'Status':<15}")
print("-" * 50)

for name, result in results.items():
    accuracy = result['accuracy']
    status = "Excellent" if accuracy > 0.8 else "Good" if accuracy > 0.7 else "Fair"
    print(f"{name:<25} {accuracy:<10.4f} {status:<15}")

# Add tuned models if available
if 'rf_tuned_accuracy' in locals():
    print(f"{'Random Forest (Tuned)':<25} {rf_tuned_accuracy:<10.4f} {'Excellent':<15}")
if 'xgb_tuned_accuracy' in locals():
    print(f"{'XGBoost (Tuned)':<25} {xgb_tuned_accuracy:<10.4f} {'Excellent':<15}")

# Best model
all_accuracies = [(name, result['accuracy']) for name, result in results.items()]
if 'rf_tuned_accuracy' in locals():
    all_accuracies.append(('Random Forest (Tuned)', rf_tuned_accuracy))
if 'xgb_tuned_accuracy' in locals():
    all_accuracies.append(('XGBoost (Tuned)', xgb_tuned_accuracy))

best_model = max(all_accuracies, key=lambda x: x[1])
print(f"\nBEST PERFORMING MODEL: {best_model[0]}")
print(f"Accuracy: {best_model[1]:.4f}")

# Recommendations
print(f"\nRECOMMENDATIONS:")
print(f"1. The {best_model[0]} model shows the best performance for this dataset.")
print(f"2. Tree-based models (Random Forest, XGBoost) significantly outperform linear models.")
print(f"3. Elevation is the most important feature for forest cover prediction.")
print(f"4. The dataset is well-balanced, making it suitable for classification.")
print(f"5. Hyperparameter tuning can provide modest improvements in performance.")

print(f"\n" + "="*60)
print("ANALYSIS COMPLETED SUCCESSFULLY!")
print("="*60)


I w